# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iskateley,67.6803,53.1512,-18.88,97,100,1.79,RU,1707167032
1,1,albany,42.6001,-73.9662,3.94,41,1,1.34,US,1707166787
2,2,iqaluit,63.7506,-68.5145,-19.06,77,100,1.03,CA,1707167032
3,3,edinburgh of the seven seas,-37.0676,-12.3116,15.77,63,2,1.95,SH,1707167032
4,4,grytviken,-54.2811,-36.5092,7.48,69,87,5.67,GS,1707167033


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",size="Humidity")

# Display the map
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_weather_df = city_data_df.loc[(city_data_df["Humidity"] < 80) & (city_data_df["Cloudiness"] < 80)]

# Drop any rows with null values
filtered_weather_df.dropna()

# Display sample data
filtered_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,albany,42.6001,-73.9662,3.94,41,1,1.34,US,1707166787
3,3,edinburgh of the seven seas,-37.0676,-12.3116,15.77,63,2,1.95,SH,1707167032
7,7,ushuaia,-54.8000,-68.3000,14.81,41,0,7.20,AR,1707167033
10,10,shahr-e babak,30.1165,55.1186,2.28,21,0,1.92,IR,1707167034
15,15,caleta de carquin,-11.0925,-77.6267,26.32,71,30,4.33,PE,1707167034
...,...,...,...,...,...,...,...,...,...,...
579,579,hailey,43.5196,-114.3153,3.28,75,40,0.00,US,1707166876
580,580,bethanie,-26.4833,17.1500,28.28,29,7,5.34,NaN,1707167155
587,587,dumfries,38.5676,-77.3280,11.81,41,0,2.57,US,1707166926
588,588,canutama,-6.5339,-64.3831,29.73,69,71,1.31,BR,1707167156


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,albany,42.6001,-73.9662,3.94,41,1,1.34,US,1707166787,
3,3,edinburgh of the seven seas,-37.0676,-12.3116,15.77,63,2,1.95,SH,1707167032,
7,7,ushuaia,-54.8000,-68.3000,14.81,41,0,7.20,AR,1707167033,
10,10,shahr-e babak,30.1165,55.1186,2.28,21,0,1.92,IR,1707167034,
15,15,caleta de carquin,-11.0925,-77.6267,26.32,71,30,4.33,PE,1707167034,
17,17,keflavik,64.0049,-22.5624,-6.05,73,40,10.29,IS,1707167035,
18,18,tinogasta,-28.0632,-67.5649,31.88,33,75,2.39,AR,1707167035,
19,19,yellowknife,62.4560,-114.3525,-22.47,77,20,3.60,CA,1707167035,
20,20,pisco,-13.7000,-76.2167,30.03,58,0,7.20,PE,1707167035,
21,21,puerto natales,-51.7236,-72.4875,16.25,39,20,16.46,CL,1707167035,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
shahr-e babak - nearest hotel: هتل شهرداری
caleta de carquin - nearest hotel: Hotel Mi Choacan
keflavik - nearest hotel: Núpan Deluxe
tinogasta - nearest hotel: Hotel de Turismo
yellowknife - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
puerto natales - nearest hotel: Dorotea Patagonia Hostel
crane - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
brookings - nearest hotel: Quality Inn
bredasdorp - nearest hotel: Victoria Hotel
quatre cocos - nearest hotel: Horisun
constantia - nearest hotel: Hotel Chérica
yuma - nearest hotel: Hilton Garden Inn Yuma Pivot Point
port lincoln - nearest hotel: Boston Hotel
whitehorse - nearest hotel: Town & Mountain Hotel
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
alofi - neare

banikoara - nearest hotel: No hotel found
grande prairie - nearest hotel: Sandman Hotel Grande Prairie
bozeman - nearest hotel: Kimpton Armory Hotel Bozeman
aswan - nearest hotel: Yaseen hotel
ahvaz - nearest hotel: هتل پارس
i-n-salah - nearest hotel: باجودة
male - nearest hotel: Champa Central Hotel
rosebank - nearest hotel: City Lodge Pinelands
siyabuswa - nearest hotel: No hotel found
natal - nearest hotel: Natal Palace Hotel
sainte-marie - nearest hotel: Hôtel Select Océan Indien
piacabucu - nearest hotel: O Leão
san vicente de canete - nearest hotel: Hotel Colibrí
kutum - nearest hotel: No hotel found
nueva guinea - nearest hotel: Oasis
saint joseph - nearest hotel: Radisson Hotel and Conference Center
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
safaga - nearest hotel: Dolphin Hotel
ondjiva - nearest hotel: Águia Verde Hotel
el granada - nearest hotel: Beach House
salinopolis - nearest hotel: Hotel Salinópolis
villa del totoral - nearest hotel: No hotel found
al bawiti

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,albany,42.6001,-73.9662,3.94,41,1,1.34,US,1707166787,No hotel found
3,3,edinburgh of the seven seas,-37.0676,-12.3116,15.77,63,2,1.95,SH,1707167032,No hotel found
7,7,ushuaia,-54.8000,-68.3000,14.81,41,0,7.20,AR,1707167033,Apart Hotel Aires del Beagle
10,10,shahr-e babak,30.1165,55.1186,2.28,21,0,1.92,IR,1707167034,هتل شهرداری
15,15,caleta de carquin,-11.0925,-77.6267,26.32,71,30,4.33,PE,1707167034,Hotel Mi Choacan
...,...,...,...,...,...,...,...,...,...,...,...
579,579,hailey,43.5196,-114.3153,3.28,75,40,0.00,US,1707166876,Wood River Inn & Suites
580,580,bethanie,-26.4833,17.1500,28.28,29,7,5.34,NaN,1707167155,No hotel found
587,587,dumfries,38.5676,-77.3280,11.81,41,0,2.57,US,1707166926,Holiday Inn Hotel Dumfries
588,588,canutama,-6.5339,-64.3831,29.73,69,71,1.31,BR,1707167156,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",size="Humidity",hover_cols=["Hotel Name","Country"])


# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)